In [88]:
import pandas as pd
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import json

print('Success')

Success


In [89]:
table = pd.read_csv('/kaggle/input/tweets-cleaned-embs/Tweets_cleaned_embs.csv')

table

,Toxicity,tweet,filtered_words,embeddings
0,0,user when a father is dysfunctional and is so ...,"['father', 'dysfunctional', 'selfish', 'drags'...","[-0.18754126131534576, -0.17380239069461823, 0..."
1,0,user user thanks for lyft credit i cant use ca...,"['thanks', 'lyft', 'credit', 'cant', 'use', 'c...","[0.037999507039785385, -0.4699377417564392, 0...."
2,0,bihday your majesty,"['bihday', 'majesty']","[-0.2989206910133362, -0.08966507017612457, -0..."
3,0,model i love u take with u all the time in ur,"['model', 'love', 'u', 'take', 'u', 'time', 'ur']","[-0.1552625298500061, -0.2156916856765747, 0.5..."
4,0,factsguide society now motivation,"['factsguide', 'society', 'motivation']","[-0.4460122585296631, -0.3412211239337921, 0.6..."
...,...,...,...,...
56740,1,yous a muthafin lie LifeAsKing Pearls coreyema...,"['yous', 'muthafin', 'lie', 'LifeAsKing', 'Pea...","[-0.114252969622612, -0.47859063744544983, 0.7..."
56741,1,youve gone and broke the wrong heart baby and ...,"['youve', 'gone', 'broke', 'wrong', 'heart', '...","[-0.23869013786315918, -0.9089678525924683, 0...."
56742,1,young buck wanna eat dat nigguh like I aint fu...,"['young', 'buck', 'wanna', 'eat', 'dat', 'nigg...","[-0.38286176323890686, -1.0216779708862305, 0...."
56743,1,youu got wild bitches tellin you lies,"['youu', 'got', 'wild', 'bitches', 'tellin', '...","[-0.23785024881362915, -0.5290617346763611, 0...."


In [90]:
X = table['embeddings']
y = table['Toxicity']

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [92]:
catboost_model = CatBoostClassifier(
    iterations=1000,
    depth=3,
    random_seed=42,
    learning_rate=0.01
)

In [94]:
X_train = X_train.apply(lambda x: json.loads(x))
X_test = X_test.apply(lambda x: json.loads(x))

X_train

33047    [0.05175567790865898, -0.4703010618686676, 0.2...
44206    [-0.18828368186950684, -0.507519543170929, 0.5...
5260     [-0.7121124863624573, -0.23219157755374908, 0....
52825    [0.09844914078712463, -0.5847457647323608, 0.7...
8952     [-0.4942774176597595, -0.15479932725429535, 0....
                               ...                        
54343    [-0.11444613337516785, -0.44153183698654175, 0...
38158    [-0.4166436493396759, -0.7630411386489868, -0....
860      [-0.33099910616874695, -0.34568142890930176, 0...
15795    [-0.3723045587539673, -0.3631531298160553, 0.3...
56422    [-0.05309170112013817, -0.48099368810653687, 0...
Name: embeddings, Length: 45396, dtype: object

In [95]:
catboost_model.fit(
    X_train,
    y_train,
    verbose=500
)

0:	learn: 0.6885369	total: 103ms	remaining: 1m 42s
500:	learn: 0.2876636	total: 46.1s	remaining: 46s
999:	learn: 0.2474546	total: 1m 29s	remaining: 0us


In [96]:
dataset_test = Pool(
    data=X_test.values.tolist(),
)

predict_classes = catboost_model.predict(dataset_test)
y_pred = predict_classes

In [97]:
print(classification_report(y_test, y+_pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      6472
           1       0.90      0.86      0.88      4877

    accuracy                           0.90     11349
   macro avg       0.90      0.90      0.90     11349
weighted avg       0.90      0.90      0.90     11349



In [98]:
print('f1_score = ', f1_score(y_test, y_pred))

f1_score =  0.8809449148113306
